In [1]:
# default_exp async_data

In [2]:
#export
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
import torch.multiprocessing as mp
from fastrl.data import *
from fastrl.basic_agents import *
from fastcore.all import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import sys

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

In [3]:
# hide
from itertools import product
from nbdev.showdoc import *
from nbdev.imports import *

from fastrl.learner import *

if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# Async Experience Blocks

> Extend traditional experience blocks to run environments asynchronously.

In [10]:
# export
def template_data_fit(queue:mp.JoinableQueue=None,items:L=None,agent:BaseAgent=None,learner_cls:Learner=None,experience_block:ExperienceBlock=None,
             cancel:mp.Event=None):
#     print('here',flush=True)
    sys.stdout.flush()
    blk=IterableDataBlock(blocks=(experience_block(agent=agent)),
                          splitter=FuncSplitter(lambda x:False))
    dls=blk.dataloaders(items,device='cpu')
#     print('Starting iter',flush=True)
    sys.stdout.flush()
    while True:
        for x in dls[0]:
#             print('putting',flush=True)
            queue.put(x)
            if cancel.is_set():
                queue.put(None)
                return None

class DataFitProcess(mp.Process):
    
    @delegates(template_data_fit,but=['queue','items'])
    def __init__(self,n:int=None,start:bool=False,data_fit=None,**kwargs):
        self.n=n
        super().__init__(target=ifnone(data_fit,template_data_fit),kwargs=kwargs)
        if start:
            self.start()
#             print('starting',data_fit,template_data_fit)
        
    def termijoin(self):
        self.terminate()
        self.join()

In [5]:
# export
class MultiProcessTfm(Transform):
    def __init__(self,n:int=1,n_processes:int=1,maxsize:int=1,process_cls=DataFitProcess):
        self.n_processes=n_processes;self.process_cls=process_cls;self.n=n;self.maxsize=maxsize
        self.queue=mp.JoinableQueue(maxsize=maxsize)
        self.cancel=mp.Event()
        self.cached_items=[]
#         mp.set_start_method('spawn',force=True)
        
    def setup(self,items:TfmdSource,train_setup=False):
#         with items:
        if len(items.items)!=0 and not issubclass(items.items[0].__class__,DataFitProcess):
            self.cached_items=deepcopy(items.items)
        self.reset(items)
            
    def reset(self,items:TfmdSource,train_setup=False):
#         with items:
        self.close(items)
        self.cancel.clear()
        self.queue=mp.JoinableQueue(maxsize=self.maxsize)
        items.items=[self.process_cls(n=self.n,start=True,queue=self.queue,items=self.cached_items,cancel=self.cancel) for _ in range(self.n_processes)]

    def close(self,items:TfmdSource):
#         with items: 
        self.cancel.set()
        try:
            while not self.queue.empty():self.queue.get()
        except (ConnectionResetError,FileNotFoundError,EOFError,ConnectionRefusedError):pass
        [p.termijoin() for p in items.items if issubclass(p.__class__,DataFitProcess)]
        items.items.clear()

    def encodes(self,o):
#         print('waiting')
        s=self.queue.get()
#         print(s[0])
        return [s]

In [6]:
# export
@delegates(MultiProcessTfm)
def AsyncExperienceBlock(experience_block,agent=None,learner_cls=None,data_fit=None,n_processes=1,n=200,bs=1,**kwargs):
    process_cls=partial(
        DataFitProcess,
        agent=agent,
        learner_cls=learner_cls,
        experience_block=experience_block,
        data_fit=data_fit
    )
    
    return TransformBlock(type_tfms=[MultiProcessTfm(process_cls=process_cls,n_processes=n_processes,n=n)],dl_type=TfmdSourceDL,
                          dls_kwargs={'bs':bs,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False})

import ptan
class TestAgent(ptan.agent.BaseAgent):
    def __call__(self,s,ss):return [0]*len(s),[0]*len(s)


In [14]:
n_envs=15
for n_step_param,steps_delta_param in product(range(1,8),range(1,8)):
    block=AsyncExperienceBlock(
        experience_block=partial(FirstLastExperienceBlock,a=0,seed=0,n_steps=n_step_param,steps_delta=steps_delta_param,
                                 dls_kwargs={'bs':1,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False}),
        n_processes=1,
        n=200
    )

    blk=IterableDataBlock(blocks=(block),splitter=FuncSplitter(lambda x:False))

    envs=[gym.make('CartPole-v1') for _ in range(n_envs)]
    envs=[SeedZeroWrapper(e) for e in envs]
    exp_src=ptan.experience.ExperienceSourceFirstLast(envs, TestAgent(), gamma=0.99,steps_count=n_step_param,steps_delta=steps_delta_param)

    dls=blk.dataloaders(['CartPole-v1']*n_envs,bs=n_step_param,num_workers=0,verbose=False,
                          indexed=True,shuffle_train=False,n=40)

    fastrl_exp=[]
    ptan_exps=[]
    counter=0

    for i,(x,e) in enumerate(zip(dls[0],exp_src)):
        for i in range(x[1].shape[0]):
#             print(x)
            fastrl_exp.append(ExperienceFirstLast(*tuple(el[i].detach().cpu().numpy()[0] if not x[-2][i] or j!=3 else None for j,el in enumerate(x))))
        for ptan_e in [e]:
            fastrl_e=fastrl_exp[counter]
            test_eq(fastrl_e.state,ptan_e.state)
            test_eq(fastrl_e.last_state,ptan_e.last_state)
            test_eq(fastrl_e.reward,ptan_e.reward)
            counter+=1

In [15]:
# model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
# agent=DQNAgent(model=model)
# learn=AgentLearner(dls,agent=agent,model=model,loss_func=lambda x:0.5)
# learn.fit(10,lr=0.01,wd=1)

# Export

In [16]:
# hide
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html()

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_data.ipynb.
Converted 05b_async_data.ipynb.
Converted 06_basic_train.ipynb.
Converted 13_metrics.ipynb.
Converted 14_actorcritic.sac.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted index.ipynb.


converting: /opt/project/fastrl/nbs/05b_async_data.ipynb
